<a href="https://colab.research.google.com/github/laNaranjaja/GenAI/blob/main/Clase_RAG_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain --quiet
!pip install pypdf --quiet
!pip install faiss-cpu --quiet
!pip install langchain-google-vertexai --quiet
!pip install --upgrade --quiet langchain langchain-google-vertexai

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import re
import os
from langchain.chat_models import ChatVertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path a tu json'

In [ ]:
chat = ChatVertexAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.vertexai.ChatVertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import ChatVertexAI`.
  warn_deprecated(


In [ ]:
pdf = "/content/gdrive/MyDrive/GenAIEne2024/CCNA.pdf"
loader = PyPDFLoader(pdf)
pages = loader.load_and_split()

In [ ]:
store_name = re.findall(r'([^\/]+)\.pdf$',pdf)[0]
store_name

'CCNA'

In [ ]:
embedding = VertexAIEmbeddings()
vectorstore = FAISS.from_documents(pages, embedding=embedding)

In [ ]:
vectorstore.save_local(f"/content/gdrive/MyDrive/GenAIEne2024/{store_name}_vector_store")

In [ ]:
query = 'What is this document about?'

In [ ]:
docs = vectorstore.similarity_search(query=query, k=10)
chain = load_qa_chain(llm=chat, chain_type='map_reduce',verbose=True)

In [ ]:
chain

MapReduceDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n______________________\n{context}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatVertexAI(client=<vertexai.language_models.ChatModel object at 0x7ceed284f520>)), reduce_documents_chain=ReduceDocumentsChain(verbose=True, combine_documents_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['question', 'summaries'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['summaries'], template="Given the following extracted parts of a long document and a quest

In [ ]:
response = chain.run(input_documents=docs, question=query)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
ptg17396363xvii
OSPFv2 Interface Configuration 218
OSPFv2 Interface Configuration Example 218
Verifying OSPFv2 Interface Configuration 219
Chapter Review 221
Chapter 9 Understanding EIGRP Concepts 224
“Do I Know This Already?” Quiz 224Foundation Topics 226EIGRP and Distance Vector Routing Protocols 226
Introduction to EIGRP 226Basic Distance Vector Routing Protocol Features 227
The Concept of a Distance and a Vector 228Full Update Messages and Split Horizon 229Route Poisoning 231
EIGRP as an Advanced DV Protocol 232
EIGRP Sends Partial Update Messages, As Needed 232EIGRP Maintains Neighbor Status Using Hello 233Summary of Interior Routing Protocol Features 233
EIGRP Concepts and Operation 234
EIGRP Neighbor

In [ ]:
print(response)

 This document appears to be a collection of technical manuals or guides related to networking and routing protocols, network programmability, networking and computing terminology, LAN topics, and OSPF and EIGRP routing protocols.
